# Managing population consumption

In [1]:
import sys
import numpy as np
import pandas as pd
import altair as alt

# mapping to the modules that make the app
sys.path.insert(0, "../..")
sys.path.insert(0, "../../app")

import yaml, ssl, asyncio, pickle, os, ast

ssl._create_default_https_context = ssl._create_unverified_context
asyncio.set_event_loop_policy(asyncio.WindowsSelectorEventLoopPolicy())

import nest_asyncio
# this is required for running in a Jupyter Notebook. 
nest_asyncio.apply()

Much of the Population growth is handled in scripts inside the Azure Function

In [3]:
# mapping to the modules that make the app

from app.connectors import cmdb_graph
from app.objects import time

c = cmdb_graph.CosmosdbClient()



In [4]:
params = yaml.safe_load(open(os.path.join(os.getenv("abspath"),"app/configurations/popgrowthconfig.yaml")))
syllables = pickle.load(open(os.path.join(os.getenv("abspath"),"app/creators/specs/syllables.p"), "rb"))
username = 'notebook'

Time is updated by the `time` function. It is always running.

In [5]:
t = time.Time(c)
t.get_current_UTU()
params['currentTime'] = t.params['currentTime']
t

< time at: 2024-05-23T16:08:51.626889+00:00 UTU:15989 >

In [6]:
params

{'pop_health_requirement': 0.7,
 'pop_consumes': 2,
 'starve_damage': 0.05,
 'changing_values': ['conformity',
  'literacy',
  'aggression',
  'constitution',
  'wealth',
  'factionLoyalty'],
 'currentTime': 15989}

For this notebook, setting the `pop_health_requirement` to an arbitrary value just to examine.

## Population Resource Consumption

In [12]:
from app.functions import consumption

We're going to run a resourse query test to ensure that consumption is happening. To test, update the `objid` with a planet that has consuming pops. 

In [ ]:
objid = '7851126304352'

In [14]:
validate_resources_updated_query = f"""
g.V().has('objid',{objid}).out('has').valueMap()
"""
print(validate_resources_updated_query)
c.run_query(validate_resources_updated_query)
pre_consumption = pd.DataFrame.from_records(c.clean_nodes(c.res))
pre_consumption


g.V().has('objid','7851126304352').out('has').valueMap()



,name,objid,volume,max_volume,description,replenish_rate,userguid,objtype,id
0,organics,0528623152769,1159,1159,bilogical material that can be consumed by pops,10.0,8d5b667f-b225-4641-b499-73b77558ff86,resource,0528623152769
1,common minerals,1853839760845,90,90,Iron and other common material used in constru...,NaN,8d5b667f-b225-4641-b499-73b77558ff86,resource,1853839760845
2,rare minerals,7874781257076,63,63,"lithium, silver and other rare minerals used i...",NaN,8d5b667f-b225-4641-b499-73b77558ff86,resource,7874781257076
3,water,0025435962279,10232,10232,"H2O ready to be consumed, either frozen or in ...",NaN,8d5b667f-b225-4641-b499-73b77558ff86,resource,0025435962279


In [30]:

all_pops_query = f"""
g.V().has('label','pop').as('pop')
    .local(
        union(
            out('inhabits').as('location'),
            out('isOf').as('species')
            )
            .fold()).as('location','species')
        .path()
        .by(unfold().valueMap().fold())
"""

In [31]:
c.run_query(all_pops_query)
data = c.reduce_res(c.res)
pd.DataFrame(data)

,pop,location,species
0,"{'name': 'Tauraite Len', 'objid': '06705474577...","{'name': 'Kotlaymau', 'class': 'terrestrial', ...","{'name': 'Alman', 'objid': '3421173158043', 'c..."
1,"{'name': 'Dosmarlutla Wo', 'objid': '952518290...","{'name': 'Kotlaymau', 'class': 'terrestrial', ...","{'name': 'Alman', 'objid': '3421173158043', 'c..."
2,"{'name': 'Tauraite Kun', 'objid': '96277650603...","{'name': 'Kotlaymau', 'class': 'terrestrial', ...","{'name': 'Alman', 'objid': '3421173158043', 'c..."
3,"{'name': 'Dosmarlutla Doncruztai', 'objid': '9...","{'name': 'Kotlaymau', 'class': 'terrestrial', ...","{'name': 'Alman', 'objid': '3421173158043', 'c..."
4,"{'name': 'Dosmarlutla Ne', 'objid': '846103886...","{'name': 'Kotlaymau', 'class': 'terrestrial', ...","{'name': 'Alman', 'objid': '3421173158043', 'c..."
5,"{'name': 'Dosmarlutla Sensta', 'objid': '42711...","{'name': 'Kotlaymau', 'class': 'terrestrial', ...","{'name': 'Alman', 'objid': '3421173158043', 'c..."
6,"{'name': 'Tauraite Munfenportsi', 'objid': '07...","{'name': 'Kotlaymau', 'class': 'terrestrial', ...","{'name': 'Alman', 'objid': '3421173158043', 'c..."
7,"{'name': 'Tauraite Lenun', 'objid': '075056938...","{'name': 'Kotlaymau', 'class': 'terrestrial', ...","{'name': 'Alman', 'objid': '3421173158043', 'c..."
8,"{'name': 'Tauraite Kunhal', 'objid': '25461367...","{'name': 'Kotlaymau', 'class': 'terrestrial', ...","{'name': 'Alman', 'objid': '3421173158043', 'c..."
9,"{'name': 'Dosmarlutla Doncruztaikay', 'objid':...","{'name': 'Kotlaymau', 'class': 'terrestrial', ...","{'name': 'Alman', 'objid': '3421173158043', 'c..."


In [32]:
pops_df = pd.DataFrame([d['pop'] for d in data])
pops_df

,name,objid,conformity,literacy,aggression,constitution,health,isIn,industry,wealth,factionLoyalty,isIdle,username,objtype,id
0,Tauraite Len,0670547457716,0.460,0.386,0.588,0.482,0.7,5998552632248,0.5350,0.4605,0.512,true,BillmanLocal2,pop,0670547457716
1,Dosmarlutla Wo,9525182909324,0.570,0.483,0.597,0.454,0.7,4460306284035,0.5255,0.5042,0.392,true,BillmanLocal2,pop,9525182909324
2,Tauraite Kun,9627765060399,0.259,0.351,0.393,0.517,0.7,5998552632248,0.4550,0.4030,0.195,true,BillmanLocal2,pop,9627765060399
3,Dosmarlutla Doncruztai,9864759742409,0.505,0.586,0.513,0.472,0.7,4460306284035,0.4925,0.5392,0.387,true,BillmanLocal2,pop,9864759742409
4,Dosmarlutla Ne,8461038868646,0.445,0.512,0.491,0.401,0.7,4460306284035,0.4460,0.4790,0.404,true,BillmanLocal2,pop,8461038868646
5,Dosmarlutla Sensta,4271120015313,0.425,0.677,0.535,0.574,0.7,4460306284035,0.5545,0.6158,0.620,true,BillmanLocal2,pop,4271120015313
6,Tauraite Munfenportsi,0758035019240,0.474,0.308,0.644,0.621,0.7,5998552632248,0.6325,0.4703,0.597,true,BillmanLocal2,pop,0758035019240
7,Tauraite Lenun,0750569389689,0.482,0.526,0.472,0.664,0.7,5998552632248,0.5680,0.5470,0.495,true,BillmanLocal2,pop,0750569389689
8,Tauraite Kunhal,2546136714796,0.588,0.564,0.699,0.498,0.7,5998552632248,0.5985,0.5812,0.593,true,BillmanLocal2,pop,2546136714796
9,Dosmarlutla Doncruztaikay,8787734600817,0.353,0.488,0.700,0.381,0.7,4460306284035,0.5405,0.5142,0.094,true,BillmanLocal2,pop,8787734600817


In [33]:
species_df = pd.DataFrame([d['species'] for d in data])
species_df

,name,objid,consumes,effuses,username,conformity,aggression,literacy,constitution,objtype,id
0,Alman,3421173158043,organics,"organic waste, plastics",BillmanLocal2,0.5,0.5,0.5,0.5,species,3421173158043
1,Alman,3421173158043,organics,"organic waste, plastics",BillmanLocal2,0.5,0.5,0.5,0.5,species,3421173158043
2,Alman,3421173158043,organics,"organic waste, plastics",BillmanLocal2,0.5,0.5,0.5,0.5,species,3421173158043
3,Alman,3421173158043,organics,"organic waste, plastics",BillmanLocal2,0.5,0.5,0.5,0.5,species,3421173158043
4,Alman,3421173158043,organics,"organic waste, plastics",BillmanLocal2,0.5,0.5,0.5,0.5,species,3421173158043
5,Alman,3421173158043,organics,"organic waste, plastics",BillmanLocal2,0.5,0.5,0.5,0.5,species,3421173158043
6,Alman,3421173158043,organics,"organic waste, plastics",BillmanLocal2,0.5,0.5,0.5,0.5,species,3421173158043
7,Alman,3421173158043,organics,"organic waste, plastics",BillmanLocal2,0.5,0.5,0.5,0.5,species,3421173158043
8,Alman,3421173158043,organics,"organic waste, plastics",BillmanLocal2,0.5,0.5,0.5,0.5,species,3421173158043
9,Alman,3421173158043,organics,"organic waste, plastics",BillmanLocal2,0.5,0.5,0.5,0.5,species,3421173158043


In [34]:
locations_df = pd.DataFrame([d['location'] for d in data])
locations_df

,name,class,objid,radius,mass,orbitsDistance,orbitsId,orbitsName,isSupportsLife,isPopulated,isHomeworld,username,objtype,id
0,Kotlaymau,terrestrial,9003968629604,0.713,0,0.638,3034142012350,Lan,true,true,true,BillmanLocal2,planet,9003968629604
1,Kotlaymau,terrestrial,9003968629604,0.713,0,0.638,3034142012350,Lan,true,true,true,BillmanLocal2,planet,9003968629604
2,Kotlaymau,terrestrial,9003968629604,0.713,0,0.638,3034142012350,Lan,true,true,true,BillmanLocal2,planet,9003968629604
3,Kotlaymau,terrestrial,9003968629604,0.713,0,0.638,3034142012350,Lan,true,true,true,BillmanLocal2,planet,9003968629604
4,Kotlaymau,terrestrial,9003968629604,0.713,0,0.638,3034142012350,Lan,true,true,true,BillmanLocal2,planet,9003968629604
5,Kotlaymau,terrestrial,9003968629604,0.713,0,0.638,3034142012350,Lan,true,true,true,BillmanLocal2,planet,9003968629604
6,Kotlaymau,terrestrial,9003968629604,0.713,0,0.638,3034142012350,Lan,true,true,true,BillmanLocal2,planet,9003968629604
7,Kotlaymau,terrestrial,9003968629604,0.713,0,0.638,3034142012350,Lan,true,true,true,BillmanLocal2,planet,9003968629604
8,Kotlaymau,terrestrial,9003968629604,0.713,0,0.638,3034142012350,Lan,true,true,true,BillmanLocal2,planet,9003968629604
9,Kotlaymau,terrestrial,9003968629604,0.713,0,0.638,3034142012350,Lan,true,true,true,BillmanLocal2,planet,9003968629604


First getting the list of consumption for each pop.


In [35]:
pops_df,species_df,locations_df = consumption.all_pops_consumption(c)


We expand that dataset to break open the list of people in each pop. 

In [36]:
params

{'pop_health_requirement': 0.6,
 'pop_consumes': 2,
 'starve_damage': 0.05,
 'changing_values': ['conformity',
  'literacy',
  'aggression',
  'constitution',
  'wealth',
  'factionLoyalty'],
 'currentTime': 1006}

In [37]:
consumption_df = consumption.get_consumption_df(locations_df,species_df,params)
consumption_df = consumption.expand_consumption_df(consumption_df)
consumption_df

,location_id,consumes,pop,consumption,multi
0,9003968629604,organics,12,24,False



Then we get a list of the resources available on that location. 

In [38]:
c.run_query(consumption.make_resource_query(consumption_df))
resources = c.res
resources

[{'labels': [['location'], ['resource']],
  'objects': [{'objid': ['9003968629604'], 'name': ['Kotlaymau']},
   {'volume': [1157], 'objid': ['3216068583822'], 'name': ['organics']}]}]

In [39]:
consumption_df = consumption.tally_consumption(c,consumption_df,resources)
consumption_df

,location_id,consumes,pop,consumption,multi,available,remaining
0,9003968629604,organics,12,24,False,1157.0,1133.0


Then we update the resources by taking out the resources that exist. You can check that this has happened in the graph by looking at the location ids. This is a verification query that isn't run in the function application.

In [40]:
consumption_df.apply(lambda x: consumption.make_resource_update_query(c,x),axis=1)

0    None
dtype: object

In [41]:
c.run_query(validate_resources_updated_query)
post_consumption = pd.DataFrame.from_records(c.clean_nodes(c.res))

In [42]:
pre_consumption

,name,objid,volume,max_volume,description,replenish_rate,username,objtype,id
0,organics,3216068583822,1157,1157,bilogical material that can be consumed by pops,10.0,BillmanLocal2,resource,3216068583822
1,common minerals,3991837221458,95,95,Iron and other common material used in constru...,NaN,BillmanLocal2,resource,3991837221458
2,rare minerals,4077355551333,68,68,"lithium, silver and other rare minerals used i...",NaN,BillmanLocal2,resource,4077355551333
3,water,2652953002191,9372,9372,"H2O ready to be consumed, either frozen or in ...",NaN,BillmanLocal2,resource,2652953002191


Only the resources that are consumed will diminish, and they will stop diminishing if the level goes below `0`

In [43]:
post_consumption

,volume,name,objid,max_volume,description,replenish_rate,username,objtype,id
0,1133,organics,3216068583822,1157,bilogical material that can be consumed by pops,10.0,BillmanLocal2,resource,3216068583822
1,95,common minerals,3991837221458,95,Iron and other common material used in constru...,NaN,BillmanLocal2,resource,3991837221458
2,68,rare minerals,4077355551333,68,"lithium, silver and other rare minerals used i...",NaN,BillmanLocal2,resource,4077355551333
3,9372,water,2652953002191,9372,"H2O ready to be consumed, either frozen or in ...",NaN,BillmanLocal2,resource,2652953002191


Now for the starving. Populations without enough food will starve (health declines). Populations with < 0 health will die.

In [44]:
consumption_df

,location_id,consumes,pop,consumption,multi,available,remaining
0,9003968629604,organics,12,24,False,1157.0,1133.0


In [45]:
consumption_df[consumption_df['remaining']<=0].apply(lambda x: consumption.lower_health(c,params,x),axis=1)

0 pops will starve in nan


Series([], dtype: float64)

Looking more closely at `lower_health`
This only happens on locations that have < 0 resources. But here we're going to do it regardless

In [46]:
x = consumption_df.loc[0]
print(x)

dead_pop_nodes = []
dead_pop_ids = []
death_event_edges = []
query =f"""
g.V().has('objid','{x.location_id}').as('location').in('inhabits')
    .haslabel('pop').as('pop')
    .out('isOf').as('species')
    .path()
        .by(valueMap('objid','name'))
        .by(valueMap('name','objid','health','username'))
        .by(valueMap('name','objid','consumes'))
"""
c.run_query(query)
out = c.res
out[0]

location_id    9003968629604
consumes            organics
pop                       12
consumption               24
multi                  False
available             1157.0
remaining             1133.0
Name: 0, dtype: object


{'labels': [['location'], ['pop'], ['species']],
 'objects': [{'objid': ['9003968629604'], 'name': ['Kotlaymau']},
  {'name': ['Tauraite Len'],
   'objid': ['0670547457716'],
   'health': [0.7],
   'username': ['BillmanLocal2']},
  {'name': ['Alman'], 'objid': ['3421173158043'], 'consumes': ['organics']}]}

## Looking more closely at starvation

* Grab my testing planet, with people on it
* Lower that amount of resource on that planet to 0
* run the lower health metric
* watch the people starve and die

In [47]:
username = 'Billmanserver'
c.run_query(f"""
            g.V().haslabel('planet').has('username','{username}').has('isPopulated','true').valuemap()
    """)

planet = c.clean_node(c.res[0])
planet

{'name': 'Lecarrang',
 'class': 'terrestrial',
 'objid': '2423418653337',
 'radius': 0.54,
 'mass': 0.821,
 'orbitsDistance': 0.939,
 'orbitsId': '5618858323815',
 'orbitsName': 'Senpol',
 'isSupportsLife': 'true',
 'isPopulated': 'true',
 'isHomeworld': 'true',
 'username': 'Billmanserver',
 'objtype': 'planet',
 'id': '2423418653337'}

Grabbing that planets organics as well, and setting the amount of the organics to zero

In [48]:
username = 'Billmanserver'
c.run_query(f"""
            g.V().haslabel('planet').has('username','{username}').has('isPopulated','true').out('has').has('name','Organic').property('volume',-1)
    """)

In [49]:
consumption_df.loc[consumption_df['location_id']==planet['objid'],['remaining','available']] = -1
consumption_df

,location_id,consumes,pop,consumption,multi,available,remaining
0,9003968629604,organics,12,24,False,1157.0,1133.0


In [50]:
username = 'Billmanserver'
c.run_query(f"""
            g.V().haslabel('planet').has('username','{username}').has('isPopulated','true').out('has').valuemap()
    """)

organics = c.clean_node(c.res[0])
organics

{'volume': -1,
 'name': 'Organic',
 'objid': '5788906859522',
 'max_volume': 1040,
 'description': 'bilogical material that can be consumed by pops',
 'replenish_rate': 10,
 'username': 'Billmanserver',
 'objtype': 'resource',
 'id': '5788906859522'}

Now that the resources are gone, the population should start to starve. 

In [51]:
consumption_df.loc[consumption_df['location_id']==planet['objid']].apply(lambda x: consumption.lower_health(c,params,x),axis=1)

0 pops will starve in nan


Series([], dtype: float64)

In [52]:
health_query =f"""
    g.V().has('objid','{planet['objid']}').as('location').in('inhabits')
        .haslabel('pop').as('pop')
        .out('isOf').as('species')
        .path()
            .by(valueMap('objid','name'))
            .by(valueMap('name','objid','health','username'))
            .by(valueMap('name','objid','consumes'))
"""

In [53]:
c.run_query(health_query)
out = c.res
pd.DataFrame([i['objects'][1] for i in out])

""


Run the following cells again and again to watch the health go down. 

In [32]:
consumption_df.loc[consumption_df['location_id']==planet['objid']].apply(lambda x: consumption.lower_health(c,params,x),axis=1)

7 pops will starve in 1425359096973


0    None
dtype: object

In [33]:
c.run_query(health_query)
out = c.res
pd.DataFrame([i['objects'][1] for i in out])

,name,objid,health,username
0,[Aythakorei Or],[9427021644478],[0.5999999999999999],[Billmanserver]
1,[Aythakorei Rezche],[8742217486310],[0.5999999999999999],[Billmanserver]
2,[Aythakorei Burghuatoucal],[2128380883047],[0.5999999999999999],[Billmanserver]
3,[Aythakorei Ton],[6992584860189],[0.5999999999999999],[Billmanserver]
4,[Borgantasga Nak],[7444602298651],[0.5999999999999999],[Billmanserver]
5,[Borgantasga Bin],[7852618490329],[0.5999999999999999],[Billmanserver]
6,[Aythakorei Ziburgdiadan],[3415888692185],[0.5999999999999999],[Billmanserver]


In [46]:
consumption_df.loc[consumption_df['location_id']==planet['objid']].apply(lambda x: consumption.lower_health(c,params,x),axis=1)
c.run_query(health_query)
out = c.res
pd.DataFrame([i['objects'][1] for i in out])

7 pops will starve in 1425359096973


""
